In [2]:
import numpy as np
import pandas as pd
import re
import warnings

#Visualisation
import matplotlib.pyplot as plt
import matplotlib
from bs4 import BeautifulSoup
import seaborn as sns
from IPython.display import display
import os
os.environ["PROJ_LIB"] = r'C:\Users\blmalumi\AppData\Local\Continuum\anaconda3\pkgs\proj4-5.2.0-ha925a31_1\Library\share'
from mpl_toolkits.basemap import Basemap
from wordcloud import WordCloud, STOPWORDS

#nltk
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.util import *
from nltk import tokenize
from sklearn.feature_extraction.text import TfidfVectorizer

matplotlib.style.use('ggplot')
pd.options.mode.chained_assignment = None
warnings.filterwarnings("ignore")

%matplotlib inline

tweets = pd.read_csv('xenophobia.csv', encoding = "ISO-8859-1")


CLEANING THE DATA

In [ ]:
#Preprocessing del RT @blablabla:
tweets['tweetos'] = '' 

#add tweetos first part
for i in range(len(tweets['text'])):
    try:
        tweets['tweetos'][i] = tweets['text'].str.split(' ')[i][0]
    except AttributeError:    
        tweets['tweetos'][i] = 'other'

#Preprocessing tweetos. select tweetos contains 'RT @'
for i in range(len(tweets['text'])):
    if tweets['tweetos'].str.contains('@')[i]  == False:
        tweets['tweetos'][i] = 'other'
        
# remove URLs, RTs, and twitter handles
for i in range(len(tweets['text'])):
    tweets['text'][i] = " ".join([word for word in tweets['text'][i].split()
                                if 'http' not in word and '@' not in word and '<' not in word])


print(tweets['text'][1])

In [ ]:
# tweets['text'] = tweets['text'].apply(lambda x: re.sub('[!@$:).;,?&]', '', x.lower()))
# tweets['text'] = tweets['text'].apply(lambda x: re.sub('  ', ' ', x))
tweets['text'][1]

In [ ]:
def wordcloud(tweets,col):
    stopwords = set(STOPWORDS)
    wordcloud = WordCloud(background_color="white",stopwords=stopwords,random_state = 2016).generate(" ".join([i for i in tweets[col]]))
    plt.figure( figsize=(20,10), facecolor='k')
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.title("Good Morning Datascience+")
wordcloud(tweets,'text')

In [ ]:
# tweets['country'] = tweets['country'].apply(lambda x: x.lower())
# tweets['country'].replace('states united','united states',inplace=True)
# tweets['country'].replace('united states','usa',inplace=True)
# tweets['country'].replace('united Kingdom','uk',inplace=True)
# tweets['country'].replace('republic philippines','philippines republic',inplace=True)
# wordcloud(tweets, 'country')

In [ ]:
tweets['source'][3]

In [ ]:
tweets['source'].head()

In [ ]:
tweets_by_type = tweets.groupby(['source'])['followers_count'].sum()
print(tweets_by_type)
plt.title('Number of followers by Source', bbox={'facecolor':'0.8', 'pad':0})
tweets_by_type.transpose().plot(kind='bar',figsize=(20, 10))

In [ ]:
tweets.describe()

In [ ]:
tweets['source_new2'] = ''
labels = ['TweetCaster for Android' ,'TwitPane for Android','Twitter for iPhone','Others','Twitter for iPad','LinkedIn','Hootsuite Inc.','Twibble.io']
labels = list(labels)
for i in range(len(tweets['source'])):
    if tweets['source'][i] not in ['Twitter for Android','Instagram','Twitter Web Client','Twitter for iPhone','LinkedIn','Hootsuite Inc.','Twibble.io']:
        tweets['source_new2'][i] = 'Others'
    else:
        tweets['source_new2'][i] = tweets['source'][i] 

tweets_by_type2 = tweets.groupby(['source_new2'])['followers_count'].sum()

tweets_by_type2.rename("",inplace=True)
explode = [1, 0, 0, 0, 0, 0,0,0]
explode = list(explode)
print(tweets_by_type2.transpose())
tweets_by_type2.transpose().plot(kind='pie',figsize=(20, 15),autopct='%1.1f%%',shadow=True,explode=explode, labels=labels)
plt.legend(bbox_to_anchor=(1, 1), loc=6, borderaxespad=0.)
plt.title('Number of followers by Source bis', bbox={'facecolor':'0.8', 'pad':5})

In [ ]:
plt.figure( figsize=(20,10), facecolor='k')
m = Basemap(projection='mill',resolution=None,llcrnrlat=-90,urcrnrlat=90,llcrnrlon=-180,urcrnrlon=180)
m.etopo()
xpt,ypt = m(np.array(tweets['place_lon']),np.array(tweets['place_lat']))
lon,lat = m(xpt,ypt,inverse=True)
m.plot(xpt,ypt,'ro',markersize=np.sqrt(5)) 
plt.title('Repartition on the globe', bbox={'facecolor':'0.8', 'pad':3})
plt.show()

In [ ]:
tweets['text_lem'] = [''.join([WordNetLemmatizer().lemmatize(re.sub('[^A-Za-z]', ' ', line)) for line in lists]).strip() for lists in tweets['text']]       
vectorizer = TfidfVectorizer(max_df=0.5,max_features=10000,min_df=10,stop_words='english',use_idf=True)
X = vectorizer.fit_transform(tweets['text_lem'].str.upper())
sid = SentimentIntensityAnalyzer()
tweets['sentiment_compound_polarity']=tweets.text_lem.apply(lambda x:sid.polarity_scores(x)['compound'])
tweets['sentiment_neutral']=tweets.text_lem.apply(lambda x:sid.polarity_scores(x)['neu'])
tweets['sentiment_negative']=tweets.text_lem.apply(lambda x:sid.polarity_scores(x)['neg'])
tweets['sentiment_pos']=tweets.text_lem.apply(lambda x:sid.polarity_scores(x)['pos'])
tweets['sentiment_type']=''
tweets.loc[tweets.sentiment_compound_polarity>0,'sentiment_type']='POSITIVE'
tweets.loc[tweets.sentiment_compound_polarity==0,'sentiment_type']='NEUTRAL'
tweets.loc[tweets.sentiment_compound_polarity<0,'sentiment_type']='NEGATIVE'

In [ ]:
tweets_sentiment = tweets.groupby(['sentiment_type'])['sentiment_neutral'].count()
tweets_sentiment.rename("",inplace=True)
explode = (1, 0, 0)
plt.subplot(221)
tweets_sentiment.transpose().plot(kind='barh',figsize=(20, 20))
plt.title('Sentiment Analysis 1', bbox={'facecolor':'0.8', 'pad':0})
plt.subplot(222)
tweets_sentiment.plot(kind='pie',figsize=(20, 20),autopct='%1.1f%%',shadow=True,explode=explode)
plt.legend(bbox_to_anchor=(1, 1), loc=3, borderaxespad=0.)
plt.title('Sentiment Analysis 2', bbox={'facecolor':'0.8', 'pad':0})
plt.show()

In [ ]:
tweets[tweets.sentiment_type == 'NEGATIVE'].text.reset_index(drop = True)[0:5]

In [ ]:
new_tweets= tweets.groupby(['created_at'])['retweet_count'].sum()
new_tweets
new_tweets.transpose().plot(kind='bar',figsize=(10, 10))

In [ ]:
combi['tidy_tweet'] = combi['tidy_tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

In [ ]:
from datetime import datetime

tweets["created_at"] = pd.to_datetime(tweets["created_at"])
tweets["user_created"] = pd.to_datetime(tweets["user_created"])

tweets["user_age"] = tweets["user_created"].apply(lambda x: (datetime.now() - x).total_seconds() / 3600 / 24 / 365)


plt.hist(tweets["user_age"])
plt.title("Tweets mentioning xenophobia")
plt.xlabel("Twitter account age in years")
plt.ylabel("# of tweets")
plt.show()

In [ ]:
#fill missing value for contries

tweets['location'].fillna(tweets['location'].mode()[0], inplace=True)

In [ ]:
tweets['location'].value_counts()

In [ ]:
tweets.isnull().sum()

In [ ]:
loc_tweets= tweets.groupby(['location'])['retweet_count'].sum()
new_tweets
new_tweets.transpose().plot(kind='bar',figsize=(10, 10))

In [ ]:
frq = tweets['location'].value_counts()/tweets.shape[0]
combine = frq.loc[frq.values < 0.0005].index
print(combine)
# for cat in combine:
#     tweets['location'].replace({cat:'Others'},inplace=True) 

In [ ]:
var = list(tweets.dtypes.loc[tweets.dtypes=='object'].index)
var

In [ ]:
[tweets['location'].value_counts()]

In [ ]:
tweets['country1'] = tweets['location']
tweets['country1'].replace('Johannesburg, South Africa','South Africa',inplace=True)
tweets['country1'].replace('Cape Town South Africa','South Africa',inplace=True)
tweets['country1'].replace('Cape Town','South Africa',inplace=True)

tweets['country1'].value_counts()

In [ ]:
frq = tweets['country1'].value_counts()/tweets.shape[0]
combine = frq.loc[frq.values < 0.0004].index

for cat in combine:
    tweets['country1'].replace({cat:'Others'},inplace=True) 

In [ ]:
tweets['country1'].value_counts()


In [ ]:
new_tweets= tweets.groupby(['country1'])['retweet_count'].sum()
new_tweets
new_tweets.transpose().plot(kind='bar',figsize=(20, 10))

In [ ]:
tweets['country1'].value_counts()
